In [140]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import sidetable as stb

from scipy.stats import skew
from scipy.stats import kurtosistest
from scipy import stats
from scipy.stats import kstest

from datetime import datetime, timedelta

from datetime import date
import holidays 

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [141]:
df = pd.read_csv('data/bikes.csv', index_col = 0).reset_index(drop = True)
df.head(1)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    object 
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(10), object(2)
memory usage: 91.4+ KB


In [143]:
# Vemos las filas y columnas que tenemos:

df.shape

(730, 16)

In [144]:
# Visualizamos que no tenemos nulos:

df.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [145]:
# Comprobamos que no hay duplicados:

df.duplicated().sum()

0

In [146]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
instant,730.0,365.500000,210.877136,1.000000,183.250000,365.500000,547.750000,730.000000
yr,730.0,0.500000,0.500343,0.000000,0.000000,0.500000,1.000000,1.000000
mnth,730.0,6.526027,3.450215,1.000000,4.000000,7.000000,10.000000,12.000000
holiday,730.0,0.028767,0.167266,0.000000,0.000000,0.000000,0.000000,1.000000
weekday,730.0,2.997260,2.006161,0.000000,1.000000,3.000000,5.000000,6.000000
workingday,730.0,0.683562,0.465405,0.000000,0.000000,1.000000,1.000000,1.000000
weathersit,730.0,1.394521,0.544807,1.000000,1.000000,1.000000,2.000000,3.000000
temp,730.0,20.319259,7.506729,2.424346,13.811885,20.465826,26.880615,35.328347
atemp,730.0,23.726322,8.150308,3.953480,16.889713,24.368225,30.445775,42.044800
hum,730.0,62.765175,14.237589,0.000000,52.000000,62.625000,72.989575,97.250000


In [147]:
lista_columnas = list(df.columns)
lista_columnas

['instant',
 'dteday',
 'season',
 'yr',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'casual',
 'registered',
 'cnt']

In [148]:
#cambiamos el nombre de las columnas para que sea más comprensible.

dicc_columnas = {'instant' : 'momento',
                'dteday' : 'fecha',
                'season' : 'estacion',
                'yr' : 'año',
                'mnth' : 'mes',
                'holiday' : 'festivo',
                'weekday' : 'dia_semana',
                'workingday' : 'laboral',
                'weathersit' : 'tiempo',
                'temp' : 'temperatura',
                'atemp' : 'sensacion_termica',
                'hum' : 'humedad',
                'windspeed' : 'viento',
                'casual' : 'casual',
                'registered' : 'registrado',
                'cnt' : 'total'}

df.rename(columns = dicc_columnas, inplace = True)

In [149]:
columnas_unicos = ['estacion', 'año', 'mes', 'festivo', 'dia_semana', 'laboral', 'tiempo']

In [150]:
# Revisamos los valores únicos para las columnas seleccionadas.

for col in columnas_unicos:
    print(f'La columna {col} tiene como valores únicos: {", ".join(map(str, df[col].unique()))}.')

La columna estacion tiene como valores únicos: spring, summer, autumn, winter.
La columna año tiene como valores únicos: 0, 1.
La columna mes tiene como valores únicos: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.
La columna festivo tiene como valores únicos: 0, 1.
La columna dia_semana tiene como valores únicos: 6, 0, 1, 2, 3, 4, 5.
La columna laboral tiene como valores únicos: 0, 1.
La columna tiempo tiene como valores únicos: 2, 1, 3.


In [151]:
# Cambiamos el formato de esta columna a datetime:

df["fecha"] = df["fecha"].apply(pd.to_datetime)

/tmp/ipykernel_25184/2154209664.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["fecha"] = df["fecha"].apply(pd.to_datetime)


In [152]:
# Modificamos la columna de años ya que únicamente había 0 y 1.

df["año"]= df["fecha"].apply(lambda x: x.strftime("%Y"))


In [153]:
df.sample(5)

,momento,fecha,estacion,año,mes,festivo,dia_semana,laboral,tiempo,temperatura,sensacion_termica,humedad,viento,casual,registrado,total
500,501,2019-05-16,summer,2019,5,0,3,1,1,26.103347,29.79875,69.7917,8.208304,991,6433,7424
676,677,2019-08-11,winter,2019,11,0,4,1,1,14.439134,17.09455,33.3478,23.304945,340,4975,5315
423,424,2019-02-28,spring,2019,2,0,2,1,1,14.725847,17.67625,39.5833,12.958939,229,4134,4363
245,246,2018-03-09,autumn,2018,9,0,6,0,1,27.435847,31.66065,71.6667,12.416775,1935,2549,4484
35,36,2018-05-02,spring,2018,2,0,6,0,2,9.566653,12.15290,92.9167,10.792293,100,905,1005


In [154]:
# Nos hemos dado cuenta de que los meses de la columna "mes" no coinciden con los de "fecha". Por tanto, vamos a ajustarlos según el mes indicado fecha.

df["mes"] = df["fecha"].dt.strftime("%B")
df["mes"].unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'],
      dtype=object)

In [155]:
meses = {"January": "enero", "February" : "febrero", "March" : "marzo", "April" : "abril", "May" : "mayo", "June" : "junio", "July" : "julio",
       "August": "agosto", "September" : "septiembre", "October" : "octubre", "November" : "noviembre", "December" : "diciembre"}


df["mes"] = df["mes"].map(meses)

In [156]:
df["mes"].unique()

array(['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio',
       'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre'],
      dtype=object)

In [157]:
# Nos hemos dado cuenta de que las estaciones están cambiadas, así que vamos a ponerlas bien.

def cambiar_estacion (fecha):

    """Recibe una columna en formato datetime, y en función del mes y el día devuelve la estación del año a la que pertenece"""
    
    mes = fecha.strftime("%B")
    dia = int(fecha.strftime("%d"))
    
    if mes == "April" or mes == "May":
        return "primavera"
    
    elif (mes == "March" and dia >= 20) or (mes == "June" and dia <= 20):
        return "primavera"
    
    if mes == "July" or  mes == "August":
        return "verano"
    
    elif (mes == "June" and dia >= 21) or (mes == "September" and dia <= 22):
        return "verano"
    
    if mes == "October" or mes == "November":
        return "otoño"
    
    elif (mes == "September" and dia >= 23) or (mes == "December" and dia <= 21):
        return "otoño"
    else:
        return "invierno"


In [158]:
df["estacion"] = df["fecha"].apply(cambiar_estacion)

In [159]:
df["estacion"].unique()

array(['invierno', 'primavera', 'verano', 'otoño'], dtype=object)

In [160]:
df.sample(10)

,momento,fecha,estacion,año,mes,festivo,dia_semana,laboral,tiempo,temperatura,sensacion_termica,humedad,viento,casual,registrado,total
699,700,2019-01-12,invierno,2019,enero,0,6,0,2,12.231653,15.84520,80.6667,4.000181,951,4240,5191
348,349,2018-12-15,otoño,2018,diciembre,0,4,1,2,17.322500,20.61185,63.4167,17.958814,181,3528,3709
543,544,2019-06-28,verano,2019,junio,0,4,1,1,30.715847,33.77560,42.2500,11.500550,921,5958,6879
374,375,2019-10-01,otoño,2019,octubre,0,2,1,1,12.656536,15.94130,64.6522,12.565984,173,3425,3598
609,610,2019-02-09,invierno,2019,febrero,0,0,0,2,28.563347,32.45000,81.5000,4.292744,2613,3197,5810
427,428,2019-04-03,primavera,2019,abril,0,0,0,1,13.359153,15.15105,40.3333,22.416257,710,2713,3423
354,355,2018-12-21,otoño,2018,diciembre,0,3,1,2,17.561653,21.40085,85.8333,14.833800,107,2553,2660
129,130,2018-10-05,otoño,2018,octubre,0,2,1,1,21.832500,26.13605,48.9167,7.749957,694,4109,4803
434,435,2019-11-03,otoño,2019,noviembre,0,0,0,1,14.831299,17.98350,47.6957,14.913329,1658,3253,4911
526,527,2019-11-06,otoño,2019,noviembre,0,1,1,2,29.554153,32.98605,58.7917,13.916771,1017,5647,6664


In [161]:
#También están mal los días de la semana, así que los cambiamos. 

df["dia_semana"] = df["fecha"].dt.strftime("%w")
df.sample(10)

,momento,fecha,estacion,año,mes,festivo,dia_semana,laboral,tiempo,temperatura,sensacion_termica,humedad,viento,casual,registrado,total
362,363,2018-12-29,invierno,2018,diciembre,0,6,1,1,10.181653,13.19460,57.4167,8.000604,254,2169,2423
145,146,2018-05-26,primavera,2018,mayo,0,6,1,1,29.041653,32.73440,67.7500,13.376014,758,3919,4677
426,427,2019-03-03,invierno,2019,marzo,0,0,0,2,16.980847,20.67460,62.1250,10.792293,956,3110,4066
283,284,2018-11-10,otoño,2018,noviembre,0,6,1,2,23.233347,27.30480,80.8750,9.583814,667,3896,4563
649,650,2019-12-10,otoño,2019,diciembre,0,2,1,1,17.937500,21.65355,53.9167,15.751164,1060,6222,7282
351,352,2018-12-18,otoño,2018,diciembre,0,2,0,1,9.771653,12.27895,58.6250,11.375193,220,2211,2431
531,532,2019-06-16,primavera,2019,junio,0,0,0,1,25.898347,29.73540,50.4167,11.166689,2963,4739,7702
321,322,2018-11-18,otoño,2018,noviembre,0,0,1,1,11.240847,13.63605,41.0000,11.291711,245,3147,3392
155,156,2018-05-06,primavera,2018,mayo,0,0,0,2,26.581653,30.84020,65.2500,9.292364,1685,3221,4906
69,70,2018-11-03,otoño,2018,noviembre,0,6,1,2,12.977402,15.25000,64.9565,15.608990,247,1730,1977


In [162]:
def cambiar_dia(columna):

    """Recibe una columna tipo datetime y devuelve el día de la semana que corresponde."""
    
    dia = columna.strftime("%w")
    
    if dia == "0":
        return "domingo" 
    elif dia == "1":
        return "lunes"
    elif dia == "2":
        return "martes"
    elif dia == "3":
        return "miércoles"
    elif dia == "4":
        return "jueves"
    elif dia == "5":
        return "viernes"
    else:
        return "sábado"

In [163]:
df["dia_semana"] = df["fecha"].apply(cambiar_dia)

In [164]:
df.index

RangeIndex(start=0, stop=730, step=1)

# Mirar librería de Holiday y hacer gráficas